In [8]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import accuracy_score

In [2]:
df = pd.read_csv('clean_lending_dataset.csv')

In [3]:
df.head()

,Unnamed: 0,emp_title,loan_amnt,funded_amnt,funded_amnt_inv,installment,int_rate,annual_inc,tot_hi_cred_lim,chargeoff_within_12_mths,...,home_ownership_RENT,verification_status,pymnt_plan,zip_code,term,emp_length,mths_since_last_delinq,issue_d,earliest_cr_line,loan_status
0,0,Chef,2500,2500,2500.0,84.92,13.56,55000.0,60124.0,0.0,...,1,0,0,109,36,10,226.0,0,6453,0
1,1,Postmaster,30000,30000,30000.0,777.23,18.94,90000.0,372872.0,0.0,...,0,1,0,713,60,10,71.0,0,11506,0
2,2,Administrative,5000,5000,5000.0,180.69,17.97,59280.0,136927.0,0.0,...,0,1,0,490,36,6,226.0,0,2801,0
3,3,IT Supervisor,4000,4000,4000.0,146.51,18.94,92000.0,385183.0,0.0,...,0,1,0,985,36,10,226.0,0,4686,0
4,4,Mechanic,30000,30000,30000.0,731.78,16.14,57250.0,157548.0,0.0,...,0,0,0,212,60,10,226.0,0,6574,0


In [4]:
df.columns

Index(['Unnamed: 0', 'emp_title', 'loan_amnt', 'funded_amnt',
       'funded_amnt_inv', 'installment', 'int_rate', 'annual_inc',
       'tot_hi_cred_lim', 'chargeoff_within_12_mths', 'total_pymnt',
       'pub_rec_bankruptcies', 'dti', 'delinq_2yrs', 'addr_state', 'purpose',
       'grade_A', 'grade_B', 'grade_C', 'grade_D', 'grade_E', 'grade_F',
       'grade_G', 'sub_grade_A1', 'sub_grade_A2', 'sub_grade_A3',
       'sub_grade_A4', 'sub_grade_A5', 'sub_grade_B1', 'sub_grade_B2',
       'sub_grade_B3', 'sub_grade_B4', 'sub_grade_B5', 'sub_grade_C1',
       'sub_grade_C2', 'sub_grade_C3', 'sub_grade_C4', 'sub_grade_C5',
       'sub_grade_D1', 'sub_grade_D2', 'sub_grade_D3', 'sub_grade_D4',
       'sub_grade_D5', 'sub_grade_E1', 'sub_grade_E2', 'sub_grade_E3',
       'sub_grade_E4', 'sub_grade_E5', 'sub_grade_F1', 'sub_grade_F2',
       'sub_grade_F3', 'sub_grade_F4', 'sub_grade_F5', 'sub_grade_G1',
       'sub_grade_G2', 'sub_grade_G3', 'sub_grade_G4', 'sub_grade_G5',
       'term_ 36 

In [70]:
g = df.columns.to_series().groupby(df.dtypes).groups
{k.name: v for k, v in g.items()}['object']

Index(['emp_title'], dtype='object')

In [59]:
state_labels, state_levels = pd.factorize(df['addr_state'])
df['addr_state'] = state_labels
df['addr_state'].unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50])

In [64]:
purpose_labels, purpose_levels = pd.factorize(df['purpose'])
df['purpose'] = purpose_labels
df['purpose'].unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13])

In [68]:
sample_df = df.sample(n=200000) # 2092883 entries

X_train, X_test, y_train, y_test = train_test_split(sample_df[['loan_amnt', 'funded_amnt',
       'funded_amnt_inv', 'installment', 'int_rate', 'annual_inc',
       'tot_hi_cred_lim', 'chargeoff_within_12_mths', 'total_pymnt',
       'pub_rec_bankruptcies', 'dti', 'delinq_2yrs',
       'grade_A', 'grade_B', 'grade_C', 'grade_D', 'grade_E', 'grade_F',
       'grade_G', 'sub_grade_A1', 'sub_grade_A2', 'sub_grade_A3',
       'sub_grade_A4', 'sub_grade_A5', 'sub_grade_B1', 'sub_grade_B2',
       'sub_grade_B3', 'sub_grade_B4', 'sub_grade_B5', 'sub_grade_C1',
       'sub_grade_C2', 'sub_grade_C3', 'sub_grade_C4', 'sub_grade_C5',
       'sub_grade_D1', 'sub_grade_D2', 'sub_grade_D3', 'sub_grade_D4',
       'sub_grade_D5', 'sub_grade_E1', 'sub_grade_E2', 'sub_grade_E3',
       'sub_grade_E4', 'sub_grade_E5', 'sub_grade_F1', 'sub_grade_F2',
       'sub_grade_F3', 'sub_grade_F4', 'sub_grade_F5', 'sub_grade_G1',
       'sub_grade_G2', 'sub_grade_G3', 'sub_grade_G4', 'sub_grade_G5',
       'term_ 36 months', 'term_ 60 months', 'home_ownership_ANY',
       'home_ownership_MORTGAGE', 'home_ownership_NONE',
       'home_ownership_OTHER', 'home_ownership_OWN', 'home_ownership_RENT',
       'verification_status', 'pymnt_plan', 'zip_code', 'term', 'emp_length',
       'mths_since_last_delinq', 'issue_d', 'earliest_cr_line',
        'addr_state'
    ]], sample_df['loan_status'], test_size=0.33, random_state=42)

In [74]:
%%time
model = xgb.XGBClassifier(
    max_depth=4, # 3 seems to be fine
    nthreads=4,
    
)
model.fit(X_train, y_train)

CPU times: user 43.5 s, sys: 228 ms, total: 43.7 s
Wall time: 43.8 s


In [75]:
y_pred = model.predict(X_test)

In [79]:
accuracy_score(y_test, y_pred)

0.9376515151515151

Hyperparameter Tuning https://www.kaggle.com/babatee/intro-xgboost-classification

In [78]:
model2 = xgb.XGBClassifier(
 learning_rate =0.1,
 n_estimators=1000,
 max_depth=3,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=1,
 seed=27)

model2.fit(X_train, y_train)

KeyboardInterrupt: 

In [ ]:
y_pred2 = model2.predict(X_test)
accuracy = accuracy_score(y_test, y_pred2)